# Example to run MCMC inversion
The inversion estimates the emissions deviation from some a priori value

In [1]:
%load_ext autoreload
%autoreload 2

import pymc as pm
import matplotlib.pyplot as plt
import numpy as np
import aesara.tensor as at
from aesara import function
from pathlib import Path

from py12box_invert.paths import Paths
from py12box_invert.invert import Invert
from py12box_invert.plot import plot_mf, plot_emissions
from py12box import core

species = "CFC-11"
project_path = Paths.data / f"example/{species}"

In [2]:
inv_mcmc_zero = Invert(project_path, species, method="mcmc_lat_gradient", n_threads=4, sensitivity_freq="yearly", start_year=2000, end_year=2020,
                sensitivity_from_zero=True)

Setting up inversion for CFC-11

No units given for emissions. Assuming Gg/yr
No inital conditions file 
... assuming zero initial conditions
Compiling model and tuning lifetime...
... completed in 3 iterations
... stratospheric lifetime: 55.0
... OH lifetime: 1e12
... ocean lifetime: 1e12
... non-OH tropospheric lifetime: 1e12
... overall lifetime: 55.0
... done in 6.0050 s


/user/home/chxmr/code/py12box_invert/py12box_invert/invert.py:117: UserWarning: Can't have both a start_year and ic_years
 Setting ic_years to None.
  warnings.warn("Can't have both a start_year and ic_years\n Setting ic_years to None.")


Calculating flux sensitivity on 4 threads...
... done
Setting initial conditions to constant value
... done


In [4]:
inv_mcmc_zero.run_inversion([10, 10, 10, 10], sensitivity_from_zero=True)

Run inversion...


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [k]
>Metropolis: [x0]
>Metropolis: [x_global]
>Metropolis: [x_ic]


/user/home/chxmr/.conda/envs/pymc/lib/python3.9/site-packages/pymc/step_methods/metropolis.py:289: RuntimeWarning: overflow encountered in exp
  "accept": np.mean(np.exp(self.accept_rate_iter)),
/user/home/chxmr/.conda/envs/pymc/lib/python3.9/site-packages/pymc/step_methods/metropolis.py:289: RuntimeWarning: overflow encountered in exp
  "accept": np.mean(np.exp(self.accept_rate_iter)),
/user/home/chxmr/.conda/envs/pymc/lib/python3.9/site-packages/pymc/step_methods/metropolis.py:289: RuntimeWarning: overflow encountered in exp
  "accept": np.mean(np.exp(self.accept_rate_iter)),
/user/home/chxmr/.conda/envs/pymc/lib/python3.9/site-packages/pymc/step_methods/metropolis.py:289: RuntimeWarning: overflow encountered in exp
  "accept": np.mean(np.exp(self.accept_rate_iter)),
Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 476 seconds.


... done
Calculating outputs...
... calculating posterior ensembles
... done



In [5]:
plot_mf(inv_mcmc_zero.outputs)

alt.VConcatChart(...)

In [6]:
plot_emissions(inv_mcmc_zero.outputs)

alt.LayerChart(...)

# Reproducing analytical Bayesian version for testing

In [ ]:
inv_true = Invert(project_path, species, method="analytical_gaussian", n_threads=4, sensitivity_freq="yearly", start_year=2010, end_year=2020)
inv_true.run_inversion([70., 20., 10., 10.])

In [ ]:
inv = Invert(project_path, species, method="mcmc_analytical", n_threads=4, sensitivity_freq="monthly", start_year=2010, end_year=2020)
inv.run_inversion([70., 20., 10., 10.])